<a href="https://colab.research.google.com/github/bernardosluz/FloorPlan-Element-Detection/blob/main/notebooks/Deteccao_Planta_Baixa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [97]:
# =============================================================================
# Importações
# =============================================================================
import numpy as np
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.patches import Rectangle, Arc, Wedge
from matplotlib.lines import Line2D
from skimage.io import imread
from scipy.ndimage import (
    binary_closing, binary_opening, binary_erosion, binary_fill_holes, sobel
)
from scipy.signal import convolve2d
from skimage.measure import label, regionprops
from skimage.draw import circle_perimeter
import os
import glob

In [96]:
# =============================================================================
# CONFIGURAÇÃO DE PASTAS
# =============================================================================
PASTA_ENTRADA = 'data'   # Coloque as plantas baixas aqui
PASTA_SAIDA = 'results'       # Resultados serão salvos aqui

os.makedirs(PASTA_ENTRADA, exist_ok=True)
os.makedirs(PASTA_SAIDA, exist_ok=True)

# Extensões aceitas
EXTENSOES = ['*.png', '*.jpg', '*.jpeg']

---
## Definição de Todas as Funções

In [88]:
# =============================================================================
# 1. Binarização e inversão
# =============================================================================
def binarizar(imagem, limiar=200):
    if len(imagem.shape) == 3:
        imagem = np.dot(imagem[..., :3], [0.299, 0.587, 0.114])
    return np.where(imagem <= limiar, 0, 1).astype(np.uint8)

def inverter(imagem_binaria):
    return 1 - imagem_binaria

In [89]:
# =============================================================================
# 2. Remoção de texto
# =============================================================================
def remover_texto_por_propriedades(img_bin_invertida, area_min, area_max):
    rotulos = label(img_bin_invertida)
    propriedades = regionprops(rotulos)
    mascara_texto = np.zeros_like(img_bin_invertida, dtype=bool)

    for prop in propriedades:
        eh_pequeno = area_min < prop.area < area_max
        minr, minc, maxr, maxc = prop.bbox
        altura_bbox = maxr - minr
        largura_bbox = maxc - minc
        razao_aspecto = largura_bbox / (altura_bbox + 0.1)
        eh_formato_letra = 0.2 < razao_aspecto < 3.0
        eh_pouco_solido = prop.solidity < 0.95
        tem_buracos = prop.euler_number < 1

        if eh_pequeno and (eh_formato_letra or tem_buracos):
            coordenadas = prop.coords
            mascara_texto[coordenadas[:, 0], coordenadas[:, 1]] = True

    img_sem_texto = img_bin_invertida.copy()
    img_sem_texto[mascara_texto] = 0
    return img_sem_texto

In [90]:
# =============================================================================
# 3. Separação de linhas retas
# =============================================================================
def remover_linhas_retas(imagem, tamanho_minimo=10):
    kernel_h = np.ones((1, tamanho_minimo), dtype=np.uint8)
    kernel_v = np.ones((tamanho_minimo, 1), dtype=np.uint8)
    linhas_h = binary_opening(imagem, structure=kernel_h)
    linhas_v = binary_opening(imagem, structure=kernel_v)
    linhas_longas = np.maximum(linhas_h, linhas_v)
    img_sem_linhas = np.logical_and(imagem, np.logical_not(linhas_longas))
    return img_sem_linhas.astype(np.uint8), linhas_longas

In [91]:
# =============================================================================
# 4. Filtro de Sobel separável
# =============================================================================
def filtro_de_sobel(imagem, kernel1, kernel2):
    temp = convolve2d(imagem, kernel1.reshape(1, -1), mode='same')
    resultado = convolve2d(temp, kernel2.reshape(-1, 1), mode='same')
    return resultado

In [92]:
# =============================================================================
# 5. Transformada de Hough Circular
# =============================================================================
def transformada_de_hough_circular(bordas, raios, limiar_votos, num_angulos=360):
    altura, largura = bordas.shape
    angulos = np.linspace(0, 2 * np.pi, num_angulos, endpoint=False)
    cos_theta = np.cos(angulos)
    sin_theta = np.sin(angulos)
    y_bordas, x_bordas = np.where(bordas > 0)
    portas_encontradas = []

    for raio in raios:
        acumulador = np.zeros((altura, largura), dtype=np.int32)
        offsets_cx = (raio * cos_theta).astype(int)
        offsets_cy = (raio * sin_theta).astype(int)

        for x, y in zip(x_bordas, y_bordas):
            cx = x - offsets_cx
            cy = y - offsets_cy
            mascara = (cx >= 0) & (cx < largura) & (cy >= 0) & (cy < altura)
            for idx in np.where(mascara)[0]:
                acumulador[cy[idx], cx[idx]] += 1

        if np.max(acumulador) == 0:
            continue
        limiar_atual = max(int(limiar_votos * np.max(acumulador)), 1)
        y_picos, x_picos = np.where(acumulador >= limiar_atual)

        for yp, xp in zip(y_picos, x_picos):
            score = acumulador[yp, xp] / num_angulos
            portas_encontradas.append((yp, xp, raio, score))

    return portas_encontradas

In [93]:
# =============================================================================
# 6. Filtro de duplicatas (NMS)
# =============================================================================
def filtrar_melhores_candidatos(candidatos, dist_minima=20, qtde_max_portas=50):
    candidatos_ordenados = sorted(candidatos, key=lambda c: c[3], reverse=True)
    aceitos = []

    for candidato in candidatos_ordenados:
        cy, cx, raio, score = candidato
        eh_novo = True
        for (ay, ax, ar, ascore) in aceitos:
            if np.sqrt((cx - ax)**2 + (cy - ay)**2) < dist_minima:
                eh_novo = False
                break
        if eh_novo:
            aceitos.append(candidato)
        if len(aceitos) == qtde_max_portas:
            break
    return aceitos

In [94]:
# =============================================================================
# 7. Validação de arcos (desenhar_arcos_reais sem plot)
# =============================================================================
def validar_arcos_porta(img_arcos, img_paredes, candidatos, min_cobertura_graus=10):
    altura, largura = img_arcos.shape
    portas_validadas = []
    candidatos = sorted(candidatos, key=lambda x: x[3], reverse=True)

    for (cy, cx, r, score_original) in candidatos:
        rr, cc = circle_perimeter(cy, cx, r, shape=(altura, largura))
        if len(rr) == 0:
            continue

        pixels_na_parede = np.sum(img_paredes[rr, cc] > 0)
        if pixels_na_parede / len(rr) > 0.2:
            continue

        mascara = img_arcos[rr, cc] > 0
        if np.sum(mascara) < (len(rr) * 0.02):
            continue

        y_reais, x_reais = rr[mascara], cc[mascara]
        angulos = np.degrees(np.arctan2(y_reais - cy, x_reais - cx))
        angulos = (angulos + 360) % 360
        angulos.sort()

        if len(angulos) < 2:
            continue

        diffs = np.diff(angulos)
        gap = (angulos[0] + 360) - angulos[-1]
        diffs = np.append(diffs, gap)
        quebras = np.where(diffs > 10)[0]

        if len(quebras) == 0:
            continue

        fronteiras = np.concatenate(([-1], quebras))
        melhor, maior = None, 0

        for j in range(len(fronteiras) - 1):
            seg = angulos[fronteiras[j]+1 : fronteiras[j+1]+1]
            if len(seg) >= 2 and seg[-1] - seg[0] > maior:
                maior = seg[-1] - seg[0]
                melhor = (seg[0], seg[-1])

        if gap <= 10 and len(quebras) >= 1:
            ext_wrap = (angulos[quebras[0]] + 360) - angulos[quebras[-1]+1]
            if ext_wrap > maior:
                maior = ext_wrap
                melhor = (angulos[quebras[-1]+1], angulos[quebras[0]] + 360)

        if melhor and min_cobertura_graus < maior < 270:
            portas_validadas.append((cy, cx, r, melhor[0], melhor[1]))

    return portas_validadas

In [95]:
# =============================================================================
# 8. Transformada de Hough Linear (com NMS e segmentos reais)
# =============================================================================
def transformada_de_hough_linear(imagem_bordas, resolucao_theta=1, limiar_votos=None,
                                  tamanho_vizinhanca=15, gap_maximo=5, comprimento_minimo=10):
    altura, largura = imagem_bordas.shape
    diagonal = int(np.ceil(np.sqrt(altura**2 + largura**2)))
    rhos = np.arange(-diagonal, diagonal + 1, 1)
    thetas = np.deg2rad(np.arange(0, 180, resolucao_theta))
    num_rhos, num_thetas = len(rhos), len(thetas)
    acumulador = np.zeros((num_rhos, num_thetas), dtype=np.int32)
    cos_t, sin_t = np.cos(thetas), np.sin(thetas)

    y_bordas, x_bordas = np.where(imagem_bordas > 0)
    for i in range(len(x_bordas)):
        x, y = x_bordas[i], y_bordas[i]
        for j in range(num_thetas):
            idx = int(round(x * cos_t[j] + y * sin_t[j])) + diagonal
            if 0 <= idx < num_rhos:
                acumulador[idx, j] += 1

    if limiar_votos is None:
        limiar_votos = int(0.5 * np.max(acumulador)) if np.max(acumulador) > 0 else 1

    # NMS
    meia = tamanho_vizinhanca // 2
    cands = np.argwhere(acumulador >= limiar_votos)
    cands_ord = sorted(cands, key=lambda c: acumulador[c[0], c[1]], reverse=True)
    suprimido = np.zeros((num_rhos, num_thetas), dtype=bool)
    linhas = []

    for (ir, it) in cands_ord:
        if suprimido[ir, it]:
            continue
        linhas.append((rhos[ir], thetas[it]))
        suprimido[max(0,ir-meia):min(num_rhos,ir+meia+1),
                  max(0,it-meia):min(num_thetas,it+meia+1)] = True

    # Extrair segmentos reais
    segmentos = []
    for rho, theta in linhas:
        ct, st = np.cos(theta), np.sin(theta)
        x0, y0 = ct * rho, st * rho
        dx, dy = -st, ct
        comp = max(altura, largura)
        seg_atual, sem_hit = [], 0

        for t in range(-comp, comp):
            px, py = int(round(x0 + t*dx)), int(round(y0 + t*dy))
            if px < 0 or px >= largura or py < 0 or py >= altura:
                if len(seg_atual) >= comprimento_minimo:
                    segmentos.append((seg_atual[0], seg_atual[-1], rho, theta))
                seg_atual, sem_hit = [], 0
                continue
            if imagem_bordas[py, px] > 0:
                seg_atual.append((px, py))
                sem_hit = 0
            else:
                sem_hit += 1
                if sem_hit > gap_maximo and seg_atual:
                    if len(seg_atual) >= comprimento_minimo:
                        segmentos.append((seg_atual[0], seg_atual[-1], rho, theta))
                    seg_atual, sem_hit = [], 0

        if len(seg_atual) >= comprimento_minimo:
            segmentos.append((seg_atual[0], seg_atual[-1], rho, theta))

    return segmentos

In [ ]:
# =============================================================================
# 9. Validação cruzada arco + reta
# =============================================================================
def validar_portas_arco_e_reta(candidatos_arcos, segmentos_retas, img_fundo, raio_erro):
    portas = []
    arcos_usados, retas_usadas = set(), set()
    altura, largura = img_fundo.shape
    margem = int(0.5 * raio_erro)

    for idx_a, cand in enumerate(candidatos_arcos):
        cy, cx, raio = cand[0], cand[1], cand[2]
        raio_exp = raio + margem
        melhor_reta, melhor_dist = None, float('inf')

        for idx_r, seg in enumerate(segmentos_retas):
            (x1, y1), (x2, y2) = seg[0], seg[1]
            d1 = np.sqrt((x1-cx)**2 + (y1-cy)**2)
            d2 = np.sqrt((x2-cx)**2 + (y2-cy)**2)
            if d1 <= raio_exp or d2 <= raio_exp:
                mx, my = (x1+x2)/2, (y1+y2)/2
                dm = np.sqrt((mx-cx)**2 + (my-cy)**2)
                if dm < melhor_dist:
                    melhor_dist = dm
                    melhor_reta = (idx_r, x1, y1, x2, y2)

        if melhor_reta:
            ir, x1, y1, x2, y2 = melhor_reta
            score = cand[3] if len(cand) > 3 else 1.0
            portas.append((cy, cx, raio, score, x1, y1, x2, y2))
            arcos_usados.add(idx_a)
            retas_usadas.add(ir)

    return portas

In [ ]:
# =============================================================================
# 10. Funções de marcação
# =============================================================================
def marcar_paredes_sobel(ax, img_paredes):
    k_suave = np.array([1, 2, 1])
    k_deriv = np.array([-1, 0, 1])
    gx = filtro_de_sobel(img_paredes, k_suave, k_deriv)
    gy = filtro_de_sobel(img_paredes, k_deriv, k_suave)
    mascara = (np.abs(gx) + np.abs(gy)) > 0.01
    by, bx = np.where(mascara)
    ax.plot(bx, by, 's', color='green', markersize=2, alpha=0.8)
    return len(by)

def marcar_portas(ax, portas_confirmadas):
    for (cy, cx, raio, score, x1, y1, x2, y2) in portas_confirmadas:
        ang1 = np.degrees(np.arctan2(y1 - cy, x1 - cx))
        ang2 = np.degrees(np.arctan2(y2 - cy, x2 - cx))
        if abs(ang2 - ang1) > 180:
            if ang2 > ang1: ang1 += 360
            else: ang2 += 360
        ax.plot([x1, x2], [y1, y2], color='red', linewidth=2.5)
    return len(portas_confirmadas)

def marcar_janelas(ax, lista_janelas):
    for (cy, cx, larg, alt) in lista_janelas:
        ret = Rectangle((cx - larg//2, cy - alt//2), larg, alt,
                         linewidth=2, edgecolor='yellow', facecolor='none')
        ax.add_patch(ret)
    return len(lista_janelas)

---
## Pipeline de Processamento Completo

In [ ]:
# =============================================================================
# Pipeline: processa UMA imagem e retorna a figura com marcações
# =============================================================================
def processar_planta(caminho_imagem):
    """
    Executa todo o pipeline de detecção em uma imagem de planta baixa.

    Etapas:
    1. Carrega e binariza
    2. Remove texto
    3. Fecha e abre para isolar paredes
    4. Extrai detalhes (XOR) e separa arcos
    5. Hough Circular → candidatos a porta
    6. Filtra e valida arcos
    7. Hough Linear → segmentos retos
    8. Validação cruzada arco + reta → portas confirmadas
    9. Monta imagem final com paredes (verde), portas (vermelho)

    Retorna:
        fig: Figura matplotlib pronta para salvar
    """
    # --- 1. Carregamento ---
    imagem_original = (imread(caminho_imagem, as_gray=True) * 255).astype('uint8')
    altura, largura = imagem_original.shape

    # --- 2. Binarização ---
    imagem_bin = binarizar(imagem_original)
    imagem_inv = inverter(imagem_bin)

    # --- 3. Remoção de texto ---
    area_min = int(0.005 * max(imagem_bin.shape))
    area_max = int(max(imagem_bin.shape))
    imagem_inv = remover_texto_por_propriedades(imagem_inv, area_min, area_max)

    # --- 4. Fechamento + Abertura → paredes ---
    k_fech = int(0.005 * min(altura, largura))
    k_fech = max(k_fech, 1)
    imagem_fechada = binary_closing(imagem_inv, structure=np.ones((k_fech, k_fech), dtype=np.uint8))

    k_aber = int(0.01 * min(altura, largura))
    k_aber = max(k_aber, 1)
    kernel_ab = np.ones((k_aber, k_aber), dtype=np.uint8)
    imagem_sem_portas = binary_opening(imagem_fechada, structure=kernel_ab)
    imagem_paredes = binary_opening(imagem_inv, structure=kernel_ab)

    # --- 5. Extração de detalhes (portas) ---
    imagem_portas = imagem_fechada ^ imagem_sem_portas
    imagem_portas[imagem_portas < 0] = 0

    tam_min_arco = int(0.05 * min(altura, largura))
    tam_min_arco = max(tam_min_arco, 1)
    imagem_arcos, _ = remover_linhas_retas(imagem_portas, tam_min_arco)

    # --- 6. Hough Circular ---
    raio_min = int(0.04 * min(altura, largura))
    raio_max = int(0.12 * min(altura, largura))
    pulo = max(1, int(0.1 * raio_min))
    raios = range(raio_min, raio_max, pulo)

    candidatos_brutos = transformada_de_hough_circular(imagem_arcos, raios, 0.4)
    dist_min_filtro = max(1, (raio_min * 2) - 1)
    candidatos_portas = filtrar_melhores_candidatos(candidatos_brutos, dist_min_filtro, 50)

    # --- 7. Validação de arcos ---
    lista_validada = validar_arcos_porta(imagem_portas, imagem_paredes, candidatos_portas, 10)

    # --- 8. Hough Linear ---
    segmentos = transformada_de_hough_linear(
        imagem_arcos.astype(np.uint8),
        resolucao_theta=1,
        limiar_votos=raio_min,
        tamanho_vizinhanca=raio_min
    )

    # --- 9. Validação cruzada ---
    cands_cruzamento = [(cy, cx, r, 1.0) for (cy, cx, r, t1, t2) in lista_validada]
    portas_confirmadas = validar_portas_arco_e_reta(
        cands_cruzamento, segmentos, imagem_inv, raio_min
    )

    # --- 10. Montar figura final ---
    fig, ax = plt.subplots(figsize=(14, 14))
    ax.imshow(imagem_original, cmap='gray')

    n_paredes = marcar_paredes_sobel(ax, imagem_sem_portas)
    n_portas = marcar_portas(ax, portas_confirmadas)
    n_janelas = marcar_janelas(ax, [])  # Janelas ainda não implementadas

    legenda = [
        Line2D([0], [0], color='green', linewidth=5, label='Paredes'),
        Line2D([0], [0], color='red', linewidth=5, label='Portas'),
        Line2D([0], [0], color='yellow', linewidth=5, label='Janelas'),
    ]
    ax.legend(handles=legenda, loc='upper right', fontsize=12)
    ax.set_title(os.path.basename(caminho_imagem), fontsize=14)
    ax.axis('off')
    plt.tight_layout()

    return fig

---
## Execução em Lote

In [ ]:
# =============================================================================
# Processa todas as imagens da pasta de entrada
# =============================================================================
# Coleta todos os arquivos de imagem
arquivos = []
for ext in EXTENSOES:
    arquivos.extend(glob.glob(os.path.join(PASTA_ENTRADA, ext)))

arquivos.sort()
print(f"Imagens encontradas: {len(arquivos)}")

for i, caminho in enumerate(arquivos):
    nome = os.path.basename(caminho)
    print(f"\n[{i+1}/{len(arquivos)}] Processando: {nome}")

    try:
        fig = processar_planta(caminho)

        # Salva com mesmo nome na pasta de saída
        nome_saida = os.path.splitext(nome)[0] + '_resultado.png'
        caminho_saida = os.path.join(PASTA_SAIDA, nome_saida)
        fig.savefig(caminho_saida, dpi=150, bbox_inches='tight', pad_inches=0.1)
        plt.close(fig)  # Libera memória

        print(f"    ✓ Salvo: {nome_saida}")

    except Exception as e:
        print(f"    ✗ Erro: {e}")

print(f"\n{'='*50}")
print(f"Concluído! Resultados em: {PASTA_SAIDA}/")

Imagens encontradas: 12

[1/12] Processando: 13380.png.jpg
    ✓ Salvo: 13380.png_resultado.png

[2/12] Processando: 1563.png.jpg
    ✓ Salvo: 1563.png_resultado.png

[3/12] Processando: 4766.png.jpg
    ✓ Salvo: 4766.png_resultado.png

[4/12] Processando: 541.png.jpg
    ✓ Salvo: 541.png_resultado.png

[5/12] Processando: 5985.png.jpg
    ✓ Salvo: 5985.png_resultado.png

[6/12] Processando: 6429.png.jpg
    ✓ Salvo: 6429.png_resultado.png

[7/12] Processando: 7108.png.jpg
    ✓ Salvo: 7108.png_resultado.png

[8/12] Processando: 788.png.jpg
    ✓ Salvo: 788.png_resultado.png

[9/12] Processando: 7912.png.jpg
    ✓ Salvo: 7912.png_resultado.png

[10/12] Processando: 8684.png.jpg
    ✓ Salvo: 8684.png_resultado.png

[11/12] Processando: 9705.png.jpg
    ✓ Salvo: 9705.png_resultado.png

[12/12] Processando: 9769.png.jpg
    ✓ Salvo: 9769.png_resultado.png

Concluído! Resultados em: results/
